In [1]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import time
import matplotlib.pyplot as plt


In [2]:
from deep_audio import Directory, JSON

In [3]:
method_algo = 'melbanks'
n_audios = 40
n_segments = 50
model_algo = 'lstm'
n_rate = 24000

DATASET_PATH = f'processed/{method_algo}_{n_audios}-{n_segments}_{n_rate}.json'

inputs, targets, mapping = Directory.load_json_data(DATASET_PATH)

In [4]:
def create_dense_model():
    # build the network architecture
    model = Sequential([
        # 1st hidden layer
        LSTM(512, input_shape=[inputs.shape[1], inputs.shape[2]], return_sequences=True),
        LSTM(256),
        Dense(128, activation='relu'),
        Dropout(0.3),
        # output layer
        Dense(len(mapping), activation='softmax'),
    ])

    optimizer = Adam(learning_rate=0.0001)


    model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

    return model


In [5]:
# SPLIT DOS DADOS
random_state = 42
# for random_state in [5438, 53, 14]:
#     for _ in range(4):
# split data into train and test set
inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs,
                                                                          targets,
                                                                          test_size=0.2,
                                                                          stratify=targets,
                                                                          random_state=random_state)

inputs_train, inputs_valid, targets_train, targets_valid = train_test_split(inputs_train,
                                                                            targets_train,
                                                                            test_size=0.2,
                                                                            stratify=targets_train,
                                                                            random_state=random_state)


In [6]:
# SALVA UMA CÓPIA DOS DADOS DE TESTE

# data = {
#     "mapping": mapping,
#     "labels": targets_test,
#     "mfcc": inputs_test,
# }

# JSON.create_json_file(f'datatest/deep/datatest_{random_state}_{inputs.shape[0]}.json', data)

In [7]:
# CRIA O MODELO
model = create_dense_model()

In [8]:
# SALVA A ESTRUTURA DO MODELO

timestamp = int(time.time())

Directory.create_directory(f'models/{model_algo}/{method_algo}/{timestamp}')

JSON.create_json_file(f'models/{model_algo}/{method_algo}/{timestamp}/model_structure.json', model.to_json())

model_save_filename = f'models/{model_algo}/{method_algo}/{timestamp}/model_weight.h5'

# DECIDE QUANDO PARAR
earlystopping_cb = EarlyStopping(patience=300, restore_best_weights=True)

# SALVA OS PESOS
mdlcheckpoint_cb = ModelCheckpoint(
    model_save_filename, monitor="val_accuracy", save_best_only=True
)

In [9]:
# TREINA O MODELO
history = model.fit(inputs_train, targets_train,
                    validation_data=(inputs_valid, targets_valid),
                    epochs=10000,
                    batch_size=128,
                    callbacks=[earlystopping_cb, mdlcheckpoint_cb])

Epoch 1/10000


In [ ]:
# GERA O GRAFICO DE ACURÁCIA
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/{model_algo}/{method_algo}/{timestamp}/graph_accuracy.png')
plt.close()

# GERA O GRÁFICO DE PERCA
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/{model_algo}/{method_algo}/{timestamp}/graph_loss.png')
plt.close()

In [11]:
# PEGA A MAIOR E ACURÁCIA
higher_accuracy = model.evaluate(inputs_test, targets_test, batch_size=128)

higher_accuracy = str(int(higher_accuracy[1] * 10000)).zfill(4)

ValueError: in user code:

    /mnt/e/faculdade/projeto_pesquisa_som/venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /mnt/e/faculdade/projeto_pesquisa_som/venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /mnt/e/faculdade/projeto_pesquisa_som/venv/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /mnt/e/faculdade/projeto_pesquisa_som/venv/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /mnt/e/faculdade/projeto_pesquisa_som/venv/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3608 _call_for_each_replica
   

NameError: name 'history' is not defined

ValueError: in user code:

    /mnt/e/faculdade/projeto_pesquisa_som/venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1323 test_function  *
        return step_function(self, iterator)
    /mnt/e/faculdade/projeto_pesquisa_som/venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1314 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /mnt/e/faculdade/projeto_pesquisa_som/venv/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /mnt/e/faculdade/projeto_pesquisa_som/venv/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /mnt/e/faculdade/projeto_pesquisa_som/venv/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3608 _call_for_each_replica
  

In [ ]:
# RENOMEIA A PASTA
Directory.rename_directory(f'models/{model_algo}/{method_algo}/{timestamp}',
                           f'models/{model_algo}/{method_algo}/acc{higher_accuracy}_seed{random_state}_epochs{len(history.history["accuracy"])}_time{timestamp}')